# Clean and convert AR6 data

In [4]:
import os
import pyam
import warnings
import geopandas
import geodatasets
import ast
import string

import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import display, HTML
from textwrap import wrap

<IPython.core.display.Javascript object>

**Note**: You need to download first the AR6 Scenarios Database for R10 regions (v1.1) from the [website](https://data.ece.iiasa.ac.at/ar6/#/downloads) and add the path below.

In [5]:
filepath_to_AR6 = '.../AR6_Scenarios_Database_R10_regions_v1.1.csv'

In [6]:
common_defs = pd.read_excel('data/common-definitions-template.xlsx',sheet_name='region')

In [7]:
common_defs_r10 = common_defs[common_defs.hierarchy == 'R10']

In [8]:
r10_regions = common_defs_r10.region.to_list()

In [9]:
convert_r10 = common_defs_r10.set_index('ar6')['region'].to_dict()

In [10]:
df_ar6_meta_vet = pd.read_excel('data/AR6/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx',sheet_name='meta_Ch3vetted_withclimate')

In [11]:
df_ar6_meta_vet['SSP'] = df_ar6_meta_vet.Ssp_family.map(lambda x: x if np.isnan(x) else f'SSP{int(x)}')

In [12]:
df_ar6_meta = pd.read_excel('data/AR6/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx',sheet_name='meta')

In [13]:
df_ar6 = pd.read_csv(filepath_to_AR6).replace({'Region':convert_r10})

In [14]:
sub_vars = ['Population','Final Energy','GDP|PPP','Emissions|CO2','Secondary Energy|Electricity|Non-Biomass Renewables',
            'Secondary Energy|Electricity|Biomass','Secondary Energy|Electricity']

In [15]:
sub_df = df_ar6[df_ar6.Variable.isin(sub_vars)]

In [16]:
sub_df_temp = sub_df.drop('Unit',axis=1).set_index(['Model','Scenario','Region','Variable'])

In [17]:
final_energy = sub_df_temp.xs("Final Energy", level="Variable")
gdp = sub_df_temp.xs("GDP|PPP", level="Variable")
co2_emissions = sub_df_temp.xs("Emissions|CO2", level="Variable")
non_biomass_res_elec = sub_df_temp.xs("Secondary Energy|Electricity|Non-Biomass Renewables", level="Variable")
biomass_res_elec = sub_df_temp.xs("Secondary Energy|Electricity|Biomass", level="Variable")
elec = sub_df_temp.xs("Secondary Energy|Electricity", level="Variable")

In [18]:
intersect_res = pd.concat([non_biomass_res_elec,biomass_res_elec,elec],join='inner',axis=1).index
intersect_energy_intensity = pd.concat([final_energy,gdp],join='inner',axis=1).index
intersect_carbon_intensity = pd.concat([co2_emissions,gdp],join='inner',axis=1).index

In [19]:
df_energy_intensity = final_energy / gdp
df_energy_intensity = df_energy_intensity.loc[intersect_energy_intensity].reset_index()
df_energy_intensity['Variable'] = 'Energy intensity'

In [20]:
df_carbon_intensity = co2_emissions / final_energy
df_carbon_intensity = df_carbon_intensity.loc[intersect_carbon_intensity].reset_index()
df_carbon_intensity['Variable'] = 'Carbon intensity'

In [21]:
df_res_share = ((non_biomass_res_elec + biomass_res_elec)/elec)
df_res_share = df_res_share.loc[intersect_res]

In [22]:
df_res_share_valid = (df_res_share<=1).sum(axis=1)
df_res_share_valid_index = df_res_share_valid[df_res_share_valid>1].index

In [23]:
df_res_share_invalid = (df_res_share>1).sum(axis=1)
df_res_share_invalid_index = df_res_share_invalid[df_res_share_invalid>1].index

In [24]:
df_res_share = df_res_share.loc[df_res_share_valid_index]

In [25]:
df_res_share = df_res_share.reset_index()
df_res_share['Variable'] = 'RES share'

In [26]:
sub_df_new = pd.concat([sub_df,df_energy_intensity,df_carbon_intensity,df_res_share])

In [27]:
vars_to_print = ['Emissions|CO2','RES share','Energy intensity','Carbon intensity']

In [28]:
sub_df_new[sub_df_new.Variable.isin(vars_to_print)].to_excel('data/AR6/R10_sub.xlsx',index=None)